# Referenzdatensatz erstellen

In [49]:
# Besonderheit HASOC 2020 Daten: Zeilenumbrüche innerhalb der Tweets
# --> notwendig, die Dateien erst so aufzubereiten, dass nur ein Tweet pro Zeile steht

# HASOC 2020 Train: "..\Korpora\German_2020_hasoc\German\hasoc_2020_de_train_new.txt"
# HASOC 2020 Test: "..\Korpora\German_2020_hasoc\German\hasoc_2020_de_test_new.txt"

def rem_white(filename):
    with open(filename, mode="r", encoding="utf-16") as f:
        content = f.readlines()
        content = content[1:] # Erklärungszeile ignorieren
        newcont = []
        for i in range(len(content)):
            # Fall 1: Zeile ist komplett
            if len(content[i].split("\t")) == 5:
                newcont.append(content[i])
            # Fall 2: Zeile ist nicht komplett
            else:
                # letzter Teil einer Zeile erreicht
                if "hasoc_2020_de_" in content[i]:
                    comp_line += content[i]
                    comp_line = comp_line.replace("\n"," ")
                    comp_line += "\n"
                    newcont.append(comp_line)
                    comp_line = "" # Zeilenakkumulator zurücksetzen
                # erster Teil einer Zeile 
                elif content[i].startswith("11") :
                    comp_line = content[i]
                # mittlerer Teil einer Zeile, manchmal nur \n
                else: comp_line += content[i]
    nwfilename = filename[:len(filename)-4] + "_formatted" + ".txt"
    with open(nwfilename, mode="w", encoding="utf-8") as outfile:
        for line in newcont:
            outfile.write(line)
    return True

# Bereits formatiert, nicht nochmals durchführen
# rem_white("..\Korpora\German_2020_hasoc\German\hasoc_2020_de_train_new.txt")
# rem_white("..\Korpora\German_2020_hasoc\German\hasoc_2020_de_test_new.txt")

In [88]:
# Besonderheit des Covid19-Abusive-Datensatzes: noch nicht unterteilt in Trainings- und Testdaten

from sklearn.model_selection import train_test_split

with open("..\Korpora\german-abusive-language-covid-19-main\covid_2021_dataset.csv", mode="r", encoding="utf-8") as infile:
    content = infile.readlines()
    explanation = content[0].split("\t")
    explanation = "\t".join([explanation[0],explanation[1],explanation[2],explanation[3]]) +"\n"
    content = content[1:]
    sep_content = [entry.split("\t") for entry in content]
    tweets = [(entry[0], entry[1], entry[2]) for entry in sep_content]
    labels = [entry[3] for entry in sep_content]

    # Separate into train/test
    # insg. 4960 Tweets, laut Paper unterteilt in Trainings-Set mit 3485 Tweets (70%), Validations-Set mit 735, und Test-Set mit 740 Tweets
    # hier entsprechend der Split 70/30, Validation-Daten werden also im Zweifelsfall von dem Testset abgespalten
    traintweets, testtweets, trainlabels, testlabels = train_test_split(tweets, labels, test_size=0.3, train_size=0.7, random_state=4, stratify=labels)

    # Reassemble into filewritable content
    train = []
    test = []
    for i, tweet in enumerate(traintweets):
        train.append("\t".join([tweet[0],tweet[1],tweet[2],trainlabels[i]]))
    for j, tweet in enumerate(testtweets):
        test.append("\t".join([tweet[0],tweet[1],tweet[2],testlabels[j]]))

with open("..\Korpora\german-abusive-language-covid-19-main\covid_2021_dataset_train.txt", mode="w", encoding="utf-8") as trainout:
    trainout.write(explanation)
    for train_point in train:
        trainout.write(train_point+"\n")

with open("..\Korpora\german-abusive-language-covid-19-main\covid_2021_dataset_test.txt", mode="w", encoding="utf-8") as testout:
    testout.write(explanation)
    for test_point in test:
        testout.write(test_point+"\n")

In [24]:
# Besonderheit des GermEval2019-Datensatzes: abgeschnittene erste Zeilen in den Testdaten
# --> notwendig, die Dateien aufzubereiten

# fehlende Buchstaben am Anfang von Tweets: abgeschnitten von GermEval2019 intern beim Übergang der Daten von
# "Testdata_Subtask12" zu "GoldLabelSubtask12"; Zeilennr. sind die gleichen
# z.B. für Zeile 341:
# "enschen, die etwas auf eBay-Kleinanzeigen verticken, wohnen entweder in einer dickichten Gartenlaube oder im obersten Stock Altbau - Hinterhaus. #issso	OTHER	OTHER"


# Testdaten: mit den Originaltweets aus der Datei "germeval2019_Testdata_Subtask12.txt" (dort ohne Labels) speichern

with open("..\Korpora\GermEval-2019-Data\germeval2019GoldLabelsSubtask1_2.txt", mode="r", encoding="utf-8") as in_test:
    cont_test = in_test.readlines()
    sep_cont = [line.strip().split("\t") for line in cont_test]

# Tweets ohne abgeschnitte Anfänge einlesen
with open("..\Korpora\GermEval-2019-Data\germeval2019_Testdata_Subtask12.txt", mode="r", encoding="utf-8") as in_tweets:
    tweets = in_tweets.readlines()
    tweets = [tweet.strip() for tweet in tweets]

# Tweets mit den Labels zusammenführen
tweets_replaced = [(tweets[i],sep_cont[i][1],sep_cont[i][2]) for i in range(len(sep_cont))]

with open("..\Korpora\GermEval-2019-Data\germeval2019GoldLabelsSubtask1_2_ersetzt.txt", mode="w", encoding="utf-8") as out_test:
    for line in tweets_replaced:
        out_test.write("\t".join(line)+"\n")

In [25]:
import re

def convert_to_refcorp(filename, corp_id, mod):
    """
    GermEval-Daten, HASOC-Daten und die Daten des Covid19-Abusive-Datensatzes in ein einheitliches Format zu übertragen.
    Input: Datei mit Tabstopp-getrennten Werten (ob txt, csv oder tsv), Korpus-ID, train/test-Information
    Output: Liste von Tupeln à (Referenzkorpus-ID, Tweet, Label1, Label2)
            - ReferenzkorpusID - setzt sich zusammen aus der Korpus-ID,
                                 md_id = "11", falls es um Trainingsdaten (mod=train), "22", falls es um Testdaten (mod=test) geht
                                 und der Zeilennummer in der Ursprungsdatei;
                                 also z.B.: "01220034" - für einen Tweet der Zeile 34, aus den Testdaten des GermEval2018-Datensatzes
            - Tweet            - String des Tweets, URLs sind mit der generischen Twitter-URL "https://t.co" ersetzt
            - Label1           -
            - Label2           -
    """
    newcorp = []
    with open(filename, mode="r", encoding="utf-8") as f:
        text = f.readlines()
        
        # erste Zeile ignorieren bei Covid19 ("05") und HASOC2019 ("03")
        if corp_id == "05" or corp_id == "03": text = text[1:]

        # Bestimmen, welche Formatierungsfunktion genutzt wird
        if corp_id == "01" or corp_id == "02": form_func = format_germeval
        elif corp_id == "03" or corp_id == "04": form_func = format_hasoc
        else: form_func = format_covidabusive

        url_pattern = re.compile('https:\/\/.*?(?: |$)')

        for num, entry in enumerate(text):
            entry = entry.strip()
            tag1, tag2 = "NOT", "NOT"

            tweet, tag1, tag2 = form_func(entry, tag1, tag2)

            # URLs mit generischer Twitter-URL ersetzen
            tweet = url_pattern.sub("https://t.co ", tweet)
            tweet = tweet.strip()

            # Tweet von HTML-Resten entfernen und Emoji-Codierung mit Emojis ersetzen
            tweet = clean_tweet(tweet)

            # ID erstellen
            if mod == "train": md_id = "11"
            elif mod =="test": md_id = "22"
            id_num = f'{num+1:04d}'
            tweet_id = str(corp_id) + str(md_id) + str(id_num)
            
            # der neuen Sammlung hinzufügen
            newcorp.append((tweet_id, tweet, tag1, tag2))
    return newcorp

def format_germeval(entry, tag1, tag2):
    tweet, label1, label2 = entry.split("\t")
    if label1 == "OFFENSE": tag1 = "NEG"
    if label2 == "INSULT": tag2 = "INSOFF"
    elif label2 == "PROFANITY": tag2 = "PRFN"
    elif label2 == "ABUSE": tag2 = "HATE"
    tweet = tweet.replace("|LBR|", " ")
    return tweet, tag1, tag2

def format_covidabusive(entry, tag1, tag2):
    sep = entry.split("\t")
    tweet, l1 = sep[1], sep[3]
    tag2 = "NAN"
    if l1 == "abuse": tag1 = "NEG"
    return tweet, tag1, tag2

def format_hasoc(entry, tag1, tag2):
    sep = entry.split("\t")
    tweet, l1, l2 = sep[1], sep[2], sep[3]
    if l1 == "HOF": tag1 = "NEG"
    if l2 == "HATE": tag2 = "HATE"
    elif l2 == "OFFN": tag2 = "INSOFF"
    elif l2 == "PRFN": tag2 = "PRFN"
    return tweet, tag1, tag2



# - 
# - html-Reste: "&amp;" anstatt "&", "\''" ansatt '"' und "&gt;" anstatt ">" usw. >> in die utf-8-Formatierung überführen
def clean_tweet(tweet):
    """Emojis finden und ersetzen (gefunden: https://stackoverflow.com/questions/67507017/replace-unicode-code-point-with-actual-character-using-regex)
        und HTML-Reste entfernen.
    """
    cleaned = tweet
    # Emojis, die als Text, z.B. "<U+0001F60A>", gespeichert sind: als utf-8 formatieren
    cleaned = re.sub(r'<U\+([A-F0-9]+)>', lambda x: chr(int(x.group(1), 16)), cleaned)
    cleaned = re.sub(r"&lt;" , "<", cleaned)	 
    cleaned = re.sub(r"&gt;" , ">", cleaned)    
    cleaned = re.sub(r"&amp;" , "&", cleaned)
    cleaned = re.sub(r'\"', '"', cleaned)
    cleaned = re.sub(r'\""', '"', cleaned)
    return cleaned

In [26]:
# GermEval2018
# Train: "..\Korpora\GermEval-2018-Data-master\germeval2018.training.txt"
# Test: "..\Korpora\GermEval-2018-Data-master\germeval2018.test.txt"
germeval2018train_converted = convert_to_refcorp("..\Korpora\GermEval-2018-Data-master\germeval2018.training.txt", "01", "train")
germeval2018test_converted = convert_to_refcorp("..\Korpora\GermEval-2018-Data-master\germeval2018.test.txt", "01", "test")

# GermEval2019
# Train: "..\Korpora\GermEval-2019-Data\germeval2019.training_subtask1_2_korrigiert.txt"
# Test: "..\Korpora\GermEval-2019-Data\germeval2019GoldLabelsSubtask1_2_ersetzt.txt"
germeval2019train_converted = convert_to_refcorp("..\Korpora\GermEval-2019-Data\germeval2019.training_subtask1_2_korrigiert.txt", "02", "train")
germeval2019test_converted = convert_to_refcorp("..\Korpora\GermEval-2019-Data\germeval2019GoldLabelsSubtask1_2_ersetzt.txt", "02", "test")

In [27]:
# HASOC 2019
# Train: Korpora\german_dataset_hasoc2019\german_dataset\german_dataset.tsv
# Test: Korpora\german_dataset_hasoc2019\german_dataset\hasoc_de_test_gold.tsv
hasoc2019train_converted = convert_to_refcorp("..\Korpora\german_dataset_hasoc2019\german_dataset\german_dataset.tsv", "03", "train")
hasoc2019test_converted = convert_to_refcorp("..\Korpora\german_dataset_hasoc2019\german_dataset\hasoc_de_test_gold.tsv", "03", "test")

# HASOC 2020
# Train: Korpora\German_2020_hasoc\German\hasoc_2020_de_train_new_formatted.txt
# Test: Korpora\German_2020_hasoc\German\hasoc_2020_de_test_new_formatted.txt
hasoc2020train_converted = convert_to_refcorp("..\Korpora\German_2020_hasoc\German\hasoc_2020_de_train_new_formatted.txt", "04", "train")
hasoc2020test_converted = convert_to_refcorp("..\Korpora\German_2020_hasoc\German\hasoc_2020_de_test_new_formatted.txt", "04", "test")

In [28]:
# Covid19 Abusive

# Korpus ursprünglich: "..\Korpora\german-abusive-language-covid-19-main\covid_2021_dataset.csv"
# Train (neu): "..\Korpora\german-abusive-language-covid-19-main\covid_2021_dataset_train.txt"
# Test (neu): "..\Korpora\german-abusive-language-covid-19-main\covid_2021_dataset_test.txt"

covidabusivetrain_converted = convert_to_refcorp("..\Korpora\german-abusive-language-covid-19-main\covid_2021_dataset_train.txt", "05", "train")
covidabusivetest_converted = convert_to_refcorp("..\Korpora\german-abusive-language-covid-19-main\covid_2021_dataset_test.txt", "05", "test")

In [46]:
import random

# Referenzdatensatz zusammenstellen
# Bereits ausgeführt, nicht noch einmal ausführen
# zweimal ausgeführt (korrigierte Formatierungen und IDs) --> Shuffle hat die Reihenfolge verändert
    # (allerdings irrelevant, da alle Tweets IDs haben)

refcorp_train = germeval2018train_converted + germeval2019train_converted + hasoc2019train_converted + hasoc2020train_converted + covidabusivetrain_converted
refcorp_test = germeval2018test_converted + germeval2019test_converted + hasoc2019test_converted + hasoc2020test_converted + covidabusivetest_converted

# sehr ähnliche Tweets: einen von beiden entfernen, via Jaccard-Index und Grenzwert


def jaccard_multisets(string1, string2):
    """Den Jaccard-Index für zwei Multisets (von zwei Strings) berechnen.
       Max.-Wert: 0.5
    """

    # Strings als sog. Multisets speichern
    # Konkret: jedes vorkommende Unigram als Key, die Anzahl der Nutzungen als Value
    str1, str2 = list(string1), list(string2)
    bag1, bag2 = dict(), dict()
    set1, set2 = set(string1), set(string2)
    for ch1 in set1:
        bag1[ch1] = str1.count(ch1)
    for ch2 in set2:
        bag2[ch2] = str2.count(ch2)

    # Schnittmenge bauen
    # als Liste von Unigrammen speichern
    # bzw. nur die Gesamtlänge (schneller)
    #schnitt = []
    schnitt_len = 0
    for gram in set1:
        if gram in set2:
            l = min(bag1[gram],bag2[gram])
            #schnitt.append(list(l*gram))
            schnitt_len += l

    # Vereinigungsmenge bauen
    # Strings in Listenform kombinieren
    vereinigung = str1 + str2
    
    jaccard_index = schnitt_len / len(vereinigung)
    return jaccard_index


# Train und Testdaten zusammenführen, um alle Tweets global auf Ähnlichkeit zu testen
# sehr ähnliche Tweets: einen von beiden entfernen (evtl. manueller Vergleich)

def collect_duplicates(corpus):
    duplicates = []
    for i in range(len(corpus)):
        if i % 10 == 0: print("i",i)
        for j in range(len(corpus)):
            #if j % 100 == 0: print("j",j)
            if i == j: continue
            else:
                jacc = jaccard_multisets(corpus[i][1],corpus[j][1])
                if jacc >= 0.4: duplicates.append((corpus[i],corpus[j]))
    return duplicates

refcorp_gsmt = refcorp_train + refcorp_test # insg. 28596 Tweets
refcorp_gsmt_HATE = [entry for entry in refcorp_gsmt if entry[3] == "HATE"] # insg. 3060 Tweets

#jaccard_sim_tweets_HATE = collect_duplicates(refcorp_gsmt_HATE)

'''random.shuffle(refcorp_train) 
random.shuffle(refcorp_test)

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Train.txt", mode="w", encoding="utf-8") as reftrainout:
    reftrainout.write("corpus_id\ttweet\tbinarylabel\tfinelabel\n")
    for reftweet in refcorp_train:
        reftrainout.write("\t".join(reftweet)+"\n")
    
with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Test.txt", mode="w", encoding="utf-8") as reftestout:
    reftestout.write("corpus_id\ttweet\tbinarylabel\tfinelabel\n")
    for reftweet in refcorp_test:
        reftestout.write("\t".join(reftweet)+"\n")'''

i 0
i 10
i 20
i 30
i 40
i 50
i 60
i 70
i 80
i 90
i 100
i 110
i 120
i 130
i 140
i 150
i 160
i 170
i 180
i 190
i 200
i 210
i 220
i 230
i 240
i 250
i 260
i 270
i 280
i 290
i 300
i 310
i 320
i 330
i 340
i 350
i 360
i 370
i 380
i 390
i 400
i 410
i 420
i 430
i 440
i 450
i 460
i 470
i 480
i 490
i 500
i 510
i 520
i 530
i 540
i 550
i 560
i 570
i 580
i 590
i 600
i 610
i 620
i 630
i 640
i 650
i 660
i 670
i 680
i 690
i 700
i 710
i 720
i 730
i 740
i 750
i 760
i 770
i 780
i 790
i 800
i 810
i 820
i 830
i 840
i 850
i 860
i 870
i 880
i 890
i 900
i 910
i 920
i 930
i 940
i 950
i 960
i 970
i 980
i 990
i 1000
i 1010
i 1020
i 1030
i 1040
i 1050
i 1060
i 1070
i 1080
i 1090
i 1100
i 1110
i 1120
i 1130
i 1140
i 1150
i 1160
i 1170
i 1180
i 1190
i 1200
i 1210
i 1220
i 1230
i 1240
i 1250
i 1260
i 1270
i 1280
i 1290
i 1300
i 1310
i 1320
i 1330
i 1340
i 1350
i 1360
i 1370
i 1380
i 1390
i 1400
i 1410
i 1420
i 1430
i 1440
i 1450
i 1460
i 1470
i 1480
i 1490
i 1500
i 1510
i 1520
i 1530
i 1540
i 1550
i 1560
i 1570
i 158

'random.shuffle(refcorp_train) \nrandom.shuffle(refcorp_test)\n\nwith open("..\\Korpora\\Referenzdatensatz_HateSpeech_Deutsch\\RefKorpHateSpeechDe_Train.txt", mode="w", encoding="utf-8") as reftrainout:\n    reftrainout.write("corpus_id\ttweet\tbinarylabel\tfinelabel\n")\n    for reftweet in refcorp_train:\n        reftrainout.write("\t".join(reftweet)+"\n")\n    \nwith open("..\\Korpora\\Referenzdatensatz_HateSpeech_Deutsch\\RefKorpHateSpeechDe_Test.txt", mode="w", encoding="utf-8") as reftestout:\n    reftestout.write("corpus_id\ttweet\tbinarylabel\tfinelabel\n")\n    for reftweet in refcorp_test:\n        reftestout.write("\t".join(reftweet)+"\n")'

In [109]:
# Ausschnittdatensätze (Train, Test) erstellen, in dem nur die Einträge mit dem feinen Label "HATE" vorkommen
# Bereits erstellt, nicht nochmals ausführen

#with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Test.txt", mode="r", encoding="utf-8") as hatetrainin:
#    all_cont = hatetrainin.readlines()
#    all_cont = all_cont[1:]
#    sep_cont = [entry.strip().split("\t") for entry in all_cont]
#    hate = []
#    for tweet in sep_cont:
#        if tweet[3] == "HATE": hate.append(tweet)

#with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Test_HATE.txt", mode="w", encoding="utf-8") as hatetrainout:
#    for line in hate:
#        hatetrainout.write("\t".join(line)+"\n")


#### Train/Test - HATE Dateien des Referenzdatensatzes als JSON-Dateien speichern

In [4]:
#with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Test_HATE.txt", mode="r", encoding="utf-8") as intxt:
#    tws = intxt.readlines()
#    conts = [elem.strip("\n").split("\t") for elem in tws]
#    cont_dicts = [{"id":elem[0], "text":elem[1], "tag1":elem[2], "tag2":elem[3]} for elem in conts]

#import json

#with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Test_HATE.json", mode="w", encoding="utf-8") as jsonout:
#    prep_cont = json.dumps(cont_dicts)
#    jsonout.write(prep_cont)

## Annotation: Logikprüfung und Formatierung

### Logikprüfung

- Anzahl Label == 0: Label "Keine" ergänzen
- Anzahl Label == 1: Label == "Keine"

- Anzahl Label > 1:
- "VVH-NS" in den Labels:
    - eine der fünf Optionen in den Labels
    - falls sonstige Label vorhanden: von Hand kontrollieren
    - kontrollieren, dass "Keine" nicht in den Labels
- "VVH" in den Labels:
    - kontrollieren, dass "Keine" nicht in den Labels
    - kontrollieren, dass eine der drei Handlungsoptionen in den Labels
    - kontrollieren, dass mind. eine Gruppe in den Labels
- Gruppen:
    - für jede Einzelgruppe kontrollieren, dass sie korrekt dem Merkmal zugeordnet wurde und dass sie mind. einmal erscheint:
    - übergeordnete Merkmale:
        - "Nationalität", 'ethnische Herkunft / "Rasse"', "Religion / Weltanschauung", "Politische Einstellung", "Geschlecht", "Anderes Merkmal"
    - Einzelgruppen gebündelt nach ihrer korrekte Zuordnung:
        - "Nationalität":
            - "Türkischstämmige Deutsche", "Marokkaner:innen", "In Deutschland lebende Ausländer:innen",
            - "Asiat:innen", "Pol:innen", "Afrikaner:innen", "Syrer:innen"
        - 'ethnische Herkunft / "Rasse"':
            - "POC", "Araber:innen"
        - "Relgion / Weltanschauung":
            - "Muslim:e/innen", "Juden/Jüdinnnen", "Christ:innen"
        - "Politische Einstellung":
            - "Die Grünen", "die SPD", "die Linke", "CDU/CSU", "AfD", "Nazis", "Islamist:innen", "Kommunist:innen", "Zionist:innen",
            - "NPD", "FDP", "Palästinenser:innen", "FridaysForFuture", "Pegida", "Anarchist:in"
        - "Geschlecht":
            - "Trans/NB-Personen", "Frauen", "Männer"
        - "Anderes Merkmal":
            - "Flüchtlinge", "Asylbewerber:innen", "Sich illegal in Deutschland aufhaltende Personen", "Migrant:innen", "Vorbestrafte",
            - "Veganer", "Senior:innen", "Lesben, Schwule, Bi", "Kinder", "Jugendliche", "Polizist:innen", "Obdachlose", "Richter:innen",
            - "Analphabet:innen", "Soldat:innen", "Behinderte"

#### Überlegungen:
- "Anderes Merkmal" feiner gliedern, Z.B. "Herkunfts- & Aufenthaltsart", "Beruf", "Alter", der Rest weiter einzeln



In [1]:
def check_anno(datensatz):
    """Annotationslogik checken und Problemfälle sammeln"""
    probleme = []
    for entry in datensatz:
        if not anno_logik_check(entry): probleme.append(entry)
    return probleme

def anno_logik_check(labelset):
    """ Die Annotationslogik checken:
        Input: Labelmenge
        Output: True (falls alles korrekt) / False (falls irgendein Problem vorliegt)
    """
    korrekt = True
    if len(labelset) == 0:
        korrekt = False
    elif (len(labelset) == 1) and ("Keine" not in labelset):
        korrekt = False
    else:
        # 1. VVH-ALLG interne Logik
        if "VVH-ALLG" in labelset:
            # Mind. eine Art der Tathandlung ggb.
            if {"Aufstachelung zu Hass", "Aufforderung zu Gewalt- oder Willkürmaßnahmen", "Angriff der Menschenwürde"} & labelset == {}: 
                korrekt = False
            # Mind. eine Gruppe genannt
            if {"Nationalität", 'ethnische Herkunft / "Rasse"', "Relgion / Weltanschauung",
                    "Politische Einstellung", "Geschlecht", "Anderes Merkmal"} & labelset == {}:
                korrekt = False
            # keine VVH-NS Labels
            if {"VVH-NS", "Billigen", "Verherrlichen", "Verharmlosen", "Leugnen", "Rechtfertigen"} & labelset != {}: 
                korrekt = False
            if "Keine" in labelset: korrekt = False

        # 2. VVH-NS interne Logik
        if "VVH-NS" in labelset:
            # Mind. eine Art der Tathandlung ggb.
            if {"Billigen", "Verherrlichen", "Verharmlosen", "Leugnen", "Rechtfertigen"} & labelset == {}: 
                korrekt = False
            # keine VVH-ALLG Labels
            if {"VVH-ALLG", "Aufstachelung zu Hass", "Aufforderung zu Gewalt- oder Willkürmaßnahmen", "Angriff der Menschenwürde"} & labelset != {}:
                korrekt = False 
            if "Keine" in labelset: korrekt = False
        
        # 3. Gruppenmerkmale - Logik (in beide Richtungen)

        nation = {"Türkischstämmige Deutsche", "Marokkaner:innen", "In Deutschland lebende Ausländer:innen",
                    "Asiat:innen", "Pol:innen", "Afrikaner:innen", "Syrer:innen"}
        herkunft = {"POC", "Araber:innen"}
        religion = {"Muslim:e/innen", "Juden/Jüdinnnen", "Christ:innen"}
        polit = {"Die Grünen", "die SPD", "die Linke", "CDU/CSU", "AfD", "Nazis", "Islamist:innen", "Kommunist:innen",
                    "Zionist:innen", "NPD", "FDP", "Palästinenser:innen", "FridaysForFuture", "Pegida", "Anarchist:in"}
        geschlecht = {"Trans/NB-Personen", "Frauen", "Männer"}
        andere = {"Flüchtlinge", "Asylbewerber:innen", "Sich illegal in Deutschland aufhaltende Personen", "Migrant:innen", "Vorbestrafte",
                    "Veganer", "Senior:innen", "Lesben, Schwule, Bi", "Kinder", "Jugendliche", "Polizist:innen", "Obdachlose",
                    "Richter:innen", "Analphabet:innen", "Soldat:innen", "Behinderte"}

        # Richtung 1: falls Gruppe vorhanden, korrektes Merkmal auch vorhanden
        for i in nation:
            if "Nationalität" not in labelset: korrekt = False
        for j in herkunft:
            if 'ethnische Herkunft / "Rasse"' not in labelset: korrekt = False
        for k in religion:
            if "Relgion / Weltanschauung" not in labelset: korrekt = False
        for l in polit:
            if "Politische Einstellung" not in labelset: korrekt = False
        for m in andere:
            if "Anderes Merkmal" not in labelset: korrekt = False

        # Richtung 2: falls Merkmal vorhanden, auch eine passende Gruppe vorhanden
        if "Nationalität" in labelset:
            if nation & labelset == {}: korrekt = False
        if 'ethnische Herkunft / "Rasse"' in labelset:
            if herkunft & labelset == {}: korrekt = False
        if "Relgion / Weltanschauung" in labelset:
            if religion & labelset == {}: korrekt = False
        if "Politische Einstellung" in labelset:
            if polit & labelset == {}: korrekt = False
        if "Geschlecht" in labelset:
            if geschlecht & labelset == {}: korrekt = False
        if "Anderes Merkmal" in labelset:
            if andere & labelset == {}: korrekt = False

    return korrekt

In [3]:
# Probleme in der Annotationslogik händisch durchgehen

import json

# Trainingsdaten
# Beispieleintrag der AnnotationsJsonDatei:
# {"id": "01112520", "data": "@SteiblBarbara @Thomas_S_Wagner @RitaKratzert Weitaus schlimmer. Heute ist es nicht mehr Dummheit.  Ein ganzes Volk ist zu (m)wutlosen Zombies dressiert worden.", "label": ["KEINE"], "tag1": "NEG", "tag2": "HATE"}
with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Train_HATE_annotiert.json", mode="r", encoding="utf-8") as trainf:
    train_anno = trainf.read()
    train_annotations = json.loads(train_anno)

print(train_annotations)

[{'id': '01112520', 'data': '@SteiblBarbara @Thomas_S_Wagner @RitaKratzert Weitaus schlimmer. Heute ist es nicht mehr Dummheit.  Ein ganzes Volk ist zu (m)wutlosen Zombies dressiert worden.', 'label': ['KEINE'], 'tag1': 'NEG', 'tag2': 'HATE'}, {'id': '01114994', 'data': 'Das Deutsche Kaiserreich soll wieder auferstehen', 'label': ['KEINE'], 'tag1': 'NEG', 'tag2': 'HATE'}, {'id': '01110544', 'data': 'Die BRD ist eine einzige Schande', 'label': ['KEINE'], 'tag1': 'NEG', 'tag2': 'HATE'}, {'id': '01114325', 'data': '@Lexie_M_ @krippmarie Die Grünen....besser kann man das Wort "überflüssig" nicht umschreiben....   Ich wünschte die grüne Brühe würde im Boden versickern.', 'label': ['Politische Einstellung', 'KEINE', 'Die Grünen'], 'tag1': 'NEG', 'tag2': 'HATE'}, {'id': '04112168', 'data': '@FakeBlondinchen @Spiegel Scheiss deutsche Politiker! Mehr gibt es wohl nicht dazu zu sagen!', 'label': ['KEINE'], 'tag1': 'NEG', 'tag2': 'HATE'}, {'id': '01110690', 'data': '@dunjahayali Also ich finde di

### Formatierung

- Kürzel mit Legende für die Labels erstellen

- in unterschiedlicher Detailgenauigkeit speichern:
    - VVH Ja/Nein, VVH-Allg Ja/Nein, VVH-NS Ja/Nein
    - Gruppe Ja/Nein, Welches Gruppenmerkmal/Keine Gruppe, Welche Gruppe genau/Keine Gruppe 
    - Handlung VVH-Allg Ja/Nein, Welche Handlung genau/Keine Handlung
    - Handlung VVH-NS Ja/Nein, Welche Handlung genau/Keine Handlung


In [37]:
tweet1 = "@Karl_Lauterbach Besser ein Amateur der lernfähig ist, als das verlauste Pack der abgefuckten SPD - SCHMAROTZER, PÄDOPHILE und DENUNZIANTEN !!!"
tweet2 = "@ThomasOppermann SPD - SCHMAROTZER,PÄDOPHILE UND DENUNZIANTEN   oder   SCHEINHEILGSTE PARTEI DEUTSCHLANDS !!!"
tweet3 = ""
tweet4 = "@spdde @hubertus_heil SPD - SCHMAROTZER, PÄDOPHILE UND DENUNZIANTEN"
tweet5 = "SPD - SCHMAROTZER, PÄDOPHILE UND DENUNZIANTEN"

jind = jaccard_multisets(tweet5, tweet4)
print(jind)

0.4017857142857143
